In [1]:
import os
import json
import nltk
from nltk.corpus import stopwords
stopwerdz = []
count = 0
with open('stopwords.txt',mode = 'r') as stopwerds:
    while 1:
        werd = stopwerds.readline()
        werd = werd.strip('\n')
        stopwerdz.append(werd)
        count = count + 1
        if count > 433:
            break
        

In [2]:
with open('wdsl.txt',mode='r',encoding = 'utf-8') as f:
    count = 0
    rows = 0
    sentences = {}
    sentence = []
    while 1:
        word = f.readline()
        rows = rows + 1
        if word != '🤩\n':
            word = word.replace('\n',' ')
            word = word.replace('\t','')
            sentence.append(word)
            if word=='endend':
                break
        else:
            sentences[count]=sentence
            sentence=[]
            count = count+1
            

In [3]:
sentences

{0: ['ministru ',
  'educație ',
  '     ',
  'sorin ',
  'câmpeanu ',
  '     ',
  'avea ',
  'spune ',
  'la ',
  'Digi24 ',
  'că ',
  'rezultat ',
  'simulare ',
  'la ',
  'evaluare ',
  'național ',
  'avea ',
  'fi ',
  'surprinzător ',
  'și ',
  'că ',
  'arăta ',
  'cât ',
  'de ',
  'bine ',
  'sine ',
  'avea ',
  'mobiliza ',
  'elev ',
  'de ',
  'clasă ',
  'a ',
  'opta ',
  'pentru ',
  'acest ',
  'examen ',
  '     ',
  'deși ',
  'avea ',
  'fi ',
  'fi ',
  'acuză ',
  'legat ',
  'de ',
  '     ',
  '   ',
  ' '],
 1: [' ',
  '   ',
  'aproape ',
  '     ',
  'de ',
  'doză ',
  'de ',
  'vaccin ',
  'astrazeneca ',
  'vrea ',
  'ajunge ',
  'duminică ',
  'la ',
  'institut ',
  'național ',
  'de ',
  'cercetare ',
  'dezvoltare ',
  '     ',
  '     ',
  'cantacuzino ',
  '     ',
  '     ',
  'transport ',
  'fi ',
  'asigurat ',
  'de ',
  'firmă ',
  'producător ',
  '     ',
  'iar ',
  'doză ',
  'de ',
  'vaccin ',
  'vrea ',
  'fi ',
  'adus ',
  'în ',


In [4]:
stopwerdz

['a',
 'abia',
 'acea',
 'aceasta',
 'această',
 'aceea',
 'aceeasi',
 'acei',
 'aceia',
 'acel',
 'acela',
 'acelasi',
 'acele',
 'acelea',
 'acest',
 'acesta',
 'aceste',
 'acestea',
 'acestei',
 'acestia',
 'acestui',
 'aceşti',
 'aceştia',
 'acolo',
 'acord',
 'acum',
 'adica',
 'ai',
 'aia',
 'aibă',
 'aici',
 'aiurea',
 'al',
 'ala',
 'alaturi',
 'ale',
 'alea',
 'alt',
 'alta',
 'altceva',
 'altcineva',
 'alte',
 'altfel',
 'alti',
 'altii',
 'altul',
 'am',
 'anume',
 'apoi',
 'ar',
 'are',
 'as',
 'asa',
 'asemenea',
 'asta',
 'astazi',
 'astea',
 'astfel',
 'astăzi',
 'asupra',
 'atare',
 'atat',
 'atata',
 'atatea',
 'atatia',
 'ati',
 'atit',
 'atita',
 'atitea',
 'atitia',
 'atunci',
 'au',
 'avea',
 'avem',
 'aveţi',
 'avut',
 'azi',
 'aş',
 'aşadar',
 'aţi',
 'b',
 'ba',
 'bine',
 'bucur',
 'bună',
 'c',
 'ca',
 'cam',
 'cand',
 'capat',
 'care',
 'careia',
 'carora',
 'caruia',
 'cat',
 'catre',
 'caut',
 'ce',
 'cea',
 'ceea',
 'cei',
 'ceilalti',
 'cel',
 'cele',
 'ce

In [5]:
stw = stopwords.words('romanian')
stw.extend(stopwerdz)

In [6]:
filtered_sentences={}
for key in sentences.keys():  
    filtered_sentence=[]
    for w in sentences[key]:
        filtered_word=''
        for letter in w:
            if (letter.isalpha()):
                filtered_word+=letter
            else:
                if(letter == '~'):
                    filtered_word+=' '
        if filtered_word not in stw and filtered_word != '' and len(filtered_word) >= 2: 
            filtered_sentence.append(filtered_word) 
    filtered_sentences[key]=filtered_sentence

In [7]:
filtered_sentences

{0: ['ministru',
  'educație',
  'sorin',
  'câmpeanu',
  'spune',
  'Digi',
  'rezultat',
  'simulare',
  'evaluare',
  'național',
  'surprinzător',
  'arăta',
  'sine',
  'mobiliza',
  'elev',
  'clasă',
  'opta',
  'examen',
  'deși',
  'acuză',
  'legat'],
 1: ['aproape',
  'doză',
  'vaccin',
  'astrazeneca',
  'vrea',
  'ajunge',
  'duminică',
  'institut',
  'național',
  'cercetare',
  'dezvoltare',
  'cantacuzino',
  'transport',
  'asigurat',
  'firmă',
  'producător',
  'doză',
  'vaccin',
  'vrea',
  'adus',
  'bucurești',
  'cale'],
 2: ['lider',
  'PNL',
  'ludovic',
  'orban',
  'transmite',
  'avertisment',
  'dur',
  'sâmbătă',
  'ministru',
  'lansa',
  'atac',
  'adresă',
  'premier',
  'florin',
  'cîțu',
  'atrage',
  'atenție',
  'ministru',
  'ataca',
  'prim',
  'ministru',
  'trebui',
  'bagaj',
  'făcut',
  'demisie',
  'scris'],
 3: ['premier',
  'florin',
  'cîțu',
  'declara',
  'decizie',
  'agenție',
  'standard',
  'Poor',
  'modificare',
  'perspectivă

In [8]:
def convert_to_string(werds):
    return ' '.join(werds)

In [9]:
for key in filtered_sentences.keys():
    filtered_sentences[key]=convert_to_string(filtered_sentences[key])

In [10]:
print(filtered_sentences[0])


ministru educație sorin câmpeanu spune Digi rezultat simulare evaluare național surprinzător arăta sine mobiliza elev clasă opta examen deși acuză legat


In [11]:
import csv
with open('filtered_output2', 'w') as csv_file:  
    writer = csv.writer(csv_file,delimiter='\t')
    for key, value in filtered_sentences.items():
       writer.writerow([value])